In [2]:
import numpy as np
import pandas as pd

In [6]:
onion = pd.read_csv('../data/theonion_1682119956.csv')
wldnws = pd.read_csv('../data/worldnews_1682119970.csv')
print(onion.shape), print(wldnws.shape)

(6000, 99)
(5989, 94)


(None, None)